In [1]:
import numpy as np
import pandas as pd

In [2]:
train = pd.read_csv('/kaggle/input/past-data-2/train_final_2.csv')
test = pd.read_csv('/kaggle/input/past-data-2/test_final_2.csv')

In [3]:
train = train.drop(columns=['by_encoded'],axis=1)
test = test.drop(columns=['by_encoded'],axis=1)

In [4]:
X = train.drop(columns = ['winner_id','match id','team1_id','team2_id'])
y = train['winner_id']

In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier



# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
from sklearn.model_selection import train_test_split, cross_val_score, KFold
cat_model = CatBoostClassifier(verbose=0,iterations=500, depth=2, learning_rate=0.01, l2_leaf_reg=20,loss_function='Logloss', random_state=42)


kf = KFold(n_splits=5, shuffle=True, random_state=42)
cross_val_scores = cross_val_score(cat_model, X_train, y_train, cv=kf, scoring='accuracy')

cat_model.fit(X_train, y_train)

y_pred = cat_model.predict(X_test)
train_pred = cat_model.predict(X_train)

# Evaluate the model

print(f'Cross-Validation Scores: {cross_val_scores}')
print(f'Mean Cross-Validation Accuracy: {np.mean(cross_val_scores)}')
accuracy = accuracy_score(y_test, y_pred)
print(f'test accuracy: {accuracy}')

accuracy = accuracy_score(y_train, train_pred)
print(f'train accuracy: {accuracy}')

Cross-Validation Scores: [0.76315789 0.75       0.75       0.74834437 0.70860927]
Mean Cross-Validation Accuracy: 0.7440223074241896
test accuracy: 0.7736842105263158
train accuracy: 0.7862796833773087


In [7]:
gbm_model = GradientBoostingClassifier(loss='log_loss', learning_rate=0.01, n_estimators=100,max_depth=2,verbose=0)
gbm_model.fit(X_train, y_train)

y_pred = gbm_model.predict(X_test)
train_pred = gbm_model.predict(X_train)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f'test accuracy: {accuracy}')

accuracy = accuracy_score(y_train, train_pred)
print(f'train accuracy: {accuracy}')

test accuracy: 0.7736842105263158
train accuracy: 0.7823218997361477


In [8]:
import xgboost as xgb
xgb_model = xgb.XGBClassifier(objective='binary:logistic',min_child_weight =6,gamma =9, eval_metric='logloss') 
xgb_model.fit(X_train, y_train)

y_pred = xgb_model.predict(X_test)
train_pred = xgb_model.predict(X_train)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f'test accuracy: {accuracy}')

accuracy = accuracy_score(y_train, train_pred)
print(f'train accuracy: {accuracy}')

test accuracy: 0.7578947368421053
train accuracy: 0.7902374670184696


In [9]:
import lightgbm as lgb
lgb_model = lgb.LGBMClassifier(boosting_type='gbdt',metric = 'binary_logloss',learning_rate= 0.001, max_depth= -1, num_leaves= 5, feature_fraction= 0.7, bagging_fraction= 1, bagging_freq= 30, n_jobs=5, n_estimators=900, verbose= -1)
lgb_model.fit(X_train, y_train)

y_pred = lgb_model.predict(X_test)
train_pred = lgb_model.predict(X_train)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f'test accuracy: {accuracy}')

accuracy = accuracy_score(y_train, train_pred)
print(f'train accuracy: {accuracy}')

test accuracy: 0.7526315789473684
train accuracy: 0.7849604221635884


In [10]:
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.metrics import accuracy_score
from sklearn.ensemble import VotingClassifier
import lightgbm as lgb


cat_model = CatBoostClassifier(verbose=0,iterations=500, depth=2, learning_rate=0.01, l2_leaf_reg=20,loss_function='Logloss', random_state=42)
gbm_model = GradientBoostingClassifier(loss='log_loss', learning_rate=0.01, n_estimators=200,max_depth=2,verbose=0)
xgb_model = xgb.XGBClassifier(objective='binary:logistic',min_child_weight =6,gamma =9, eval_metric='logloss') 
lgb_model = lgb.LGBMClassifier(boosting_type='gbdt',metric = 'binary_logloss',learning_rate= 0.001, max_depth= -1, num_leaves= 5, feature_fraction= 0.7, bagging_fraction= 1, bagging_freq= 30, n_jobs=5, n_estimators=900, verbose= -1)

ensemble_model = VotingClassifier(estimators=[
   ('gbm', gbm_model),
    ('xgboost', xgb_model),
    ('catboost', cat_model),
    ('lgbm', lgb_model)
], voting='soft') # 'soft' uses predicted probabilities; 'hard' uses predicted class labels

# K-Fold Cross Validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)
cross_val_scores = cross_val_score(ensemble_model, X_train, y_train, cv=kf, scoring='accuracy')

# Train ensemble model
ensemble_model.fit(X_train, y_train)

# Predict on test set
y_pred = ensemble_model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f'Cross-Validation Scores: {cross_val_scores}')
print(f'Mean Cross-Validation Accuracy: {np.mean(cross_val_scores)}')
print(f'test accuracy: {accuracy}')

Cross-Validation Scores: [0.73026316 0.75       0.78289474 0.74834437 0.72847682]
Mean Cross-Validation Accuracy: 0.7479958173579645
test accuracy: 0.7736842105263158


In [ ]:
from sklearn.ensemble import StackingClassifier


cat_model = CatBoostClassifier(verbose=0,iterations=200, depth=2, learning_rate=0.01, l2_leaf_reg=20,loss_function='Logloss', random_state=42)
gbm_model = GradientBoostingClassifier(loss='log_loss', learning_rate=0.01, n_estimators=800,max_depth=1,verbose=0)
xgb_model = xgb.XGBClassifier(objective='binary:logistic',min_child_weight =6,gamma =9, eval_metric='logloss') 
lgb_model = lgb.LGBMClassifier(boosting_type='gbdt',metric = 'binary_logloss',learning_rate= 0.001, max_depth= -1, num_leaves= 5, feature_fraction= 0.7, bagging_fraction= 1, bagging_freq= 30, n_jobs=5, n_estimators=900, verbose= -1)

# stacking_clf = StackingClassifier(
#     estimators=[
#         ('xgb', xgb_model),
#         ('catboost', xgb_model),
#         ('gbm', xgb_model),
#         ('lgbm', xgb_model),
#         ('xgb1',xgb_model),
#         ('xgb2',xgb_model)
#     ],
#     final_estimator=xgb_model)
stacking_clf = StackingClassifier(
    estimators=[
        ('xgb', cat_model),
        ('catboost', cat_model),
        ('gbm', cat_model),
        ('lgbm', cat_model)
    ],
    final_estimator=cat_model)


# stacking_clf = StackingClassifier(
#     estimators=[
#         ('gbm', gbm_model),
#     ('xgboost', xgb_model),
#     ('catboost', cat_model),
#     ('lgbm', lgb_model)
#     ],
#     final_estimator=cat_model)
# CatBoostClassifier(verbose=0,iterations=1000, depth=2, learning_rate=0.01, l2_leaf_reg=20,loss_function='Logloss', random_state=42 )
kf = KFold(n_splits=5, shuffle=True, random_state=42)
cross_val_scores = cross_val_score(stacking_clf, X_train, y_train, cv=kf, scoring='accuracy')

# Train ensemble model
stacking_clf.fit(X_train, y_train)

# Predict on test set
y_pred = stacking_clf.predict(X_test)
y_train_pred = stacking_clf.predict(X_train)

# Evaluate the model
test_accuracy = accuracy_score(y_test, y_pred)
print(f'Cross-Validation Scores: {cross_val_scores}')
print(f'Mean Cross-Validation Accuracy: {np.mean(cross_val_scores)}')
print(f'test accuracy: {test_accuracy}')

train_accuracy = accuracy_score(y_train, y_train_pred)
print(f'train accuracy: {train_accuracy}')

**Testing on actual test data**

In [ ]:
X_test = test.drop(columns=['match id','team1_id','team2_id'],axis=1)

In [ ]:
cat_model = CatBoostClassifier(verbose=0,iterations=500, depth=2, learning_rate=0.01, l2_leaf_reg=20,loss_function='Logloss', random_state=42)
gbm_model = GradientBoostingClassifier(loss='log_loss', learning_rate=0.01, n_estimators=200,max_depth=2,verbose=0)
xgb_model = xgb.XGBClassifier(objective='binary:logistic',min_child_weight =6,gamma =9, eval_metric='logloss') 
lgb_model = lgb.LGBMClassifier(boosting_type='gbdt',metric = 'binary_logloss',learning_rate= 0.001, max_depth= -1, num_leaves= 5, feature_fraction= 0.7, bagging_fraction= 1, bagging_freq= 30, n_jobs=5, n_estimators=900, verbose= -1)

ensemble_model = VotingClassifier(estimators=[
    ('gbm', gbm_model),
    ('xgboost', xgb_model),
    ('catboost', cat_model),
    ('lgbm', lgb_model)
], voting='soft') # 'soft' uses predicted probabilities; 'hard' uses predicted class labels

# stacking_clf = StackingClassifier(
#     estimators=[
#         ('xgb', cat_model),
#         ('catboost', cat_model),
#         ('gbm', cat_model),
#         ('lgbm', cat_model)
#     ],
#     final_estimator=cat_model)
# Train ensemble model
ensemble_model.fit(X, y)

test_cat_pred = ensemble_model.predict(X_test)
test_prob_scores = ensemble_model.predict_proba(X_test)

train_cat_pred = ensemble_model.predict(X)
train_prob_scores = ensemble_model.predict_proba(X)

In [ ]:
train_p_scores = np.amax(train_prob_scores,axis=1)
test_p_scores = np.amax(test_prob_scores,axis=1)

In [ ]:
sample_df = pd.read_csv('/kaggle/input/amex-bowl-hackathon/6645b7f8b0de7_sample_template/sample_template/submission_template_file1.csv')
sample_df

In [ ]:
df_train = pd.DataFrame()
df_train['match id'] = train['match id']
df_train['dataset_type'] = 'train'
df_train['win_pred_team'] = train_cat_pred
df_train['win_pred_score'] = train_p_scores
df_train['train_algorithm'] = 'gbm;xgboost;catboost;lgbm'
df_train['is_ensemble'] = 'yes'
df_train['train_hps_trees'] = '200;100;500;900'
df_train['train_hps_depth'] = '2;3;2;-1'
df_train['train_hps_lr'] = '0.01;0.1;0.01;0.001'
df_train['team1'] = train['team1_id']
df_train['team2'] = train['team2_id']

df_test = pd.DataFrame()
df_test['match id'] = test['match id']
df_test['dataset_type'] = 'r1'
df_test['win_pred_team'] = test_cat_pred
df_test['win_pred_score'] = test_p_scores
df_test['train_algorithm'] = 'gbm;xgboost;catboost;lgbm'
df_test['is_ensemble'] = 'yes'
df_test['train_hps_trees'] = '200;100;500;900'
df_test['train_hps_depth'] = '2;3;2;-1'
df_test['train_hps_lr'] = '0.01;0.1;0.01;0.001'
df_test['team1'] = test['team1_id']
df_test['team2'] = test['team2_id']

In [ ]:
# df_train = pd.DataFrame()
# df_train['match id'] = train['match id']
# df_train['dataset_type'] = 'train'
# df_train['win_pred_team'] = train_cat_pred
# df_train['win_pred_score'] = train_p_scores
# df_train['train_algorithm'] = 'catboost;catboost;catboost;catboost;catboost'
# df_train['is_ensemble'] = 'yes'
# df_train['train_hps_trees'] = '500;500;500;500;500'
# df_train['train_hps_depth'] = '2;2;2;2;2'
# df_train['train_hps_lr'] = '0.01;0.01;0.01;0.01;0.01'
# df_train['team1'] = train['team1_id']
# df_train['team2'] = train['team2_id']


# df_test = pd.DataFrame()
# df_test['match id'] = test['match id']
# df_test['dataset_type'] = 'r1'
# df_test['win_pred_team'] = test_cat_pred
# df_test['win_pred_score'] = test_p_scores
# df_test['train_algorithm'] ='catboost;catboost;catboost;catboost;catboost'
# df_test['is_ensemble'] = 'yes'
# df_test['train_hps_trees'] = '500;500;500;500;500'
# df_test['train_hps_depth'] = '2;2;2;2;2'
# df_test['train_hps_lr'] =  '0.01;0.01;0.01;0.01;0.01'
# df_test['team1'] = test['team1_id']
# df_test['team2'] = test['team2_id']

In [ ]:
df_test['win_pred_team_id'] = [df_test['team1'][i] if df_test['win_pred_team'][i] == 1 else df_test['team2'][i] for i in range(len(df_test))]
df_train['win_pred_team_id'] = [df_train['team1'][i] if df_train['win_pred_team'][i] == 1 else df_train['team2'][i] for i in range(len(df_train))]

In [ ]:
df_train.drop(columns = ['team1','team2','win_pred_team'],axis=1, inplace=True)
df_test.drop(columns = ['team1','team2','win_pred_team'],axis=1, inplace=True)

In [ ]:
columns = ['match id', 'dataset_type', 'win_pred_team_id', 'win_pred_score',
       'train_algorithm', 'is_ensemble', 'train_hps_trees', 'train_hps_depth',
       'train_hps_lr']
df_train = df_train[columns]
df_test = df_test[columns]

In [ ]:
df = pd.concat([df_test, df_train], axis=0)

In [ ]:
df.set_index('match id', inplace=True)
sample_df.set_index('match id', inplace=True)

In [ ]:
df = df.reindex(sample_df.index)
df.reset_index(inplace=True)
sample_df.reset_index(inplace=True)


In [ ]:
import pandas as pd

# Define columns
columns = ['indep_feat_id1', 'indep_feat_id2', 'indep_feat_id3',
           'indep_feat_id4', 'indep_feat_id5', 'indep_feat_id6',
           'indep_feat_id7', 'indep_feat_id8', 'indep_feat_id9',
           'indep_feat_id10']

# Create DataFrame
feat_data = pd.DataFrame(columns=columns)
feat_data['match id'] = df['match id']

i=1
for col in columns:
    feat_data[col] = i
    i+=1


In [ ]:
fil1_df = pd.concat([df, feat_data], axis=1)
fil1_df.drop(columns=['match id'],axis=1,inplace=True)
fil1_df['match id'] = df['match id']
fil1_df = fil1_df[sample_df.columns]
fil1_df['win_pred_team_id'] = fil1_df['win_pred_team_id'].astype('int64')
fil1_df.to_csv('/kaggle/working/file1_new_imputation.csv',index=False)

In [ ]:
cat_model = CatBoostClassifier(verbose=0,iterations=500, depth=2, learning_rate=0.01, l2_leaf_reg=20,loss_function='Logloss', random_state=42)
gbm_model = GradientBoostingClassifier(loss='log_loss', learning_rate=0.01, n_estimators=200,max_depth=2,verbose=0)
xgb_model = xgb.XGBClassifier(objective='binary:logistic',min_child_weight =6,gamma =9, eval_metric='logloss') 
lgb_model = lgb.LGBMClassifier(boosting_type='gbdt',metric = 'binary_logloss',learning_rate= 0.001, max_depth= -1, num_leaves= 5, feature_fraction= 0.7, bagging_fraction= 1, bagging_freq= 30, n_jobs=5, n_estimators=900, verbose= -1)

xgb_model.fit(X, y)
gbm_model.fit(X, y)
lgb_model.fit(X, y)
cat_model.fit(X, y)

# Extract Feature Importances
xgb_importance = xgb_model.feature_importances_
gbm_importance = gbm_model.feature_importances_
lgbm_importance = lgb_model.feature_importances_
catboost_importance = cat_model.feature_importances_
catboost_importance = catboost_importance/100
lgbm_importance = lgbm_importance/3600
# Average Feature Importances
feature_importances = np.mean([xgb_importance, gbm_importance, lgbm_importance, catboost_importance], axis=0)

In [ ]:
# feature_importance  = cat_model.feature_importances_
feature_importance_df = pd.DataFrame({'feature': X.columns, 'importance': feature_importances*100})
feature_importance_df = feature_importance_df.sort_values(by='importance', ascending=False)
sam = pd.read_csv('/kaggle/input/amex-bowl-hackathon/6645b7f8b0de7_sample_template/sample_template/submission_template_file2.csv')
feature_importance_df.reset_index(inplace=True)
feature_importance_df.drop(columns=['index'],axis=1,inplace=True)
feature_importance_df.rename(columns={'feature':'feat_name','importance':'model_feat_imp_train'},inplace=True)

feature_importance_df['feat_rank_train'] = feature_importance_df.index
feature_importance_df['feat_rank_train']= feature_importance_df['feat_rank_train']+1

feature_importance_df['feat_id'] = feature_importance_df['feat_rank_train']
feature_importance_df['feat_description'] = feature_importance_df['feat_name']
feature_importance_df = feature_importance_df[sam.columns]

feature_importance_df.to_csv('/kaggle/working/file2_new_imputation.csv',index=False)

In [ ]:
feature_importance_df